In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
from numpy import loadtxt
import xgboost
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import sklearn
sklearn.set_config(transform_output="pandas")
import category_encoders as ce
import graphviz
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot
from sklearn.base import clone
from sklearn.model_selection import StratifiedKFold, cross_validate

In [ ]:
df = pd.read_csv('./data/final_script.csv')
df = df.loc[:,~df.columns.str.startswith('Unnamed')]
df.head(5)

In [ ]:
df['fpw'] = df['first_party_winner'].map({'True': 1,'False' :0 })
min_df = df[['href', 'fpw', 'issue_area']]
sim1_facts_df = pd.merge(df, min_df, how="left", suffixes=(None, '_sim1_facts'), left_on='sim1_facts_href', right_on='href')
sim2_facts_df = pd.merge(sim1_facts_df, min_df, how="left", suffixes=(None, '_sim2_facts'), left_on='sim2_facts_href', right_on='href')

sim1_issue_area_df = pd.merge(sim2_facts_df, min_df, how="left", suffixes=(None, '_sim1_issue_area'), left_on='sim1_issue_area_href', right_on='href')
sim2_issue_area_df = pd.merge(sim1_issue_area_df, min_df, how="left", suffixes=(None, '_sim2_issue_area'), left_on='sim2_issue_area_href', right_on='href')

sim1_legal_question_df = pd.merge(sim2_issue_area_df, min_df, how="left", suffixes=(None, '_sim1_legal_question'), left_on='sim1_legal_question_href', right_on='href')

sim1_conclusion_df = pd.merge(sim1_legal_question_df, min_df, how="left", suffixes=(None, '_sim1_conclusion'), left_on='sim1_conclusion_href', right_on='href')

merged_df = sim1_conclusion_df
merged_df['fpw'] = merged_df[merged_df['fpw'].notna()]['fpw'].astype(int)
merged_df = merged_df[merged_df['fpw'].notna()]

In [ ]:
merged_df = merged_df[merged_df.columns.drop(list(merged_df.filter(regex='Unnamed')))]
merged_df = merged_df[merged_df.columns.drop(list(merged_df.filter(regex='href')))]
merged_df = merged_df[merged_df['term'].str.isnumeric()]
merged_df = merged_df.drop('ID', axis=1)
merged_df = merged_df.drop('name', axis=1)
merged_df = merged_df.drop('docket', axis=1)
merged_df = merged_df.drop('facts', axis=1)
merged_df = merged_df.drop('facts_len', axis=1)
merged_df = merged_df.drop('majority_vote', axis=1)
merged_df = merged_df.drop('minority_vote', axis=1)
merged_df = merged_df.drop('disposition', axis=1)
merged_df = merged_df.drop('decision_type', axis=1)
merged_df = merged_df.drop('first_party_winner', axis=1)
merged_df = merged_df.drop('legal_question', axis=1)
merged_df = merged_df.drop('facts_clean', axis=1)
merged_df = merged_df.drop('conclusion', axis=1)
merged_df['term'] = merged_df['term'].astype('float')

merged_df.columns


In [ ]:
cat_vars = ['first_party',
            'second_party',
            'issue_area',
            'first_party_entity',
            'second_party_entity',
            'judges',
            'lower_court',
            'issue_area_sim1_facts',
            'issue_area_sim2_facts',
            'issue_area_sim1_conclusion',
            'issue_area_sim1_legal_question',
            'issue_area_sim1_issue_area',
            'issue_area_sim2_issue_area']
for col in cat_vars:
    merged_df[col] = merged_df[col].astype('category')

In [ ]:
merged_df.to_csv('for_training.csv')

In [ ]:
# y = merged_df['fpw']
# X = merged_df.drop('fpw', axis=1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y)

In [ ]:
merged_df.info()

In [25]:
#https://xgboost.readthedocs.io/en/stable/python/sklearn_estimator.html#early-stopping
y = merged_df['fpw']
X = merged_df.drop('fpw', axis=1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y)
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=94)

estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42,
    enable_categorical=True,
    tree_method='hist',
    early_stopping_rounds=10
)
parameters = {
    'max_depth': range (2, 20, 5),
    'n_estimators': range(10, 85, 5),
    'learning_rate': [0.1, 0.01, 0.05],
    'booster': ['gbtree'],
    'objective': ['binary:logistic']
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    #stratified
    cv = 10,
    verbose=10
)
# grid_search.fit(X, y)

def fit_and_score(estimator, X_train, X_test, y_train, y_test):
    """Fit the estimator on the train set and score it on both sets"""
    estimator.fit(X_train, y_train, eval_set=[(X_test, y_test)])

    train_score = estimator.score(X_train, y_train)
    test_score = estimator.score(X_test, y_test)

    return estimator, train_score, test_score

results = {}

#https://xgboost.readthedocs.io/en/stable/python/sklearn_estimator.html#early-stopping
for train, test in cv.split(X, y):
    print(train)
    print(text)
    X_train = X.iloc[train]
    X_test = X.iloc[test]
    y_train = y.iloc[train]
    y_test = y.iloc[test]
    est, train_score, test_score = fit_and_score(
        clone(grid_search), X_train, X_test, y_train, y_test
    )
    results[est] = (train_score, test_score)
    print(grid_search.best_estimator_)

print("===========")
print(results)

In [ ]:
print(grid_search.best_estimator_)

In [ ]:
grid_predictions = grid_search.predict(X_test)
print(classification_report(y_test, grid_predictions))
print(confusion_matrix(y_test, grid_predictions))
# AUC as it is a function of sensitivity and specificity, the curve is insensitive to disparities in the class proportions
# https://stats.stackexchange.com/questions/235808/binary-classification-with-strongly-unbalanced-classes
print(f"roc auc: {sklearn.metrics.roc_auc_score( y_test, grid_predictions)}")

In [ ]:
print(grid_search.best_estimator_.feature_importances_)
# plot
pyplot.bar(range(len(grid_search.best_estimator_.feature_importances_)), grid_search.best_estimator_.feature_importances_)
pyplot.show()

In [ ]:
# data = xgb.DMatrix(X, label=y, enable_categorical=True)
# params = {'max_depth': 5, 'eta':3, 'objective':'binary:logistic', 'n_estimators': range(10, 100, 5), 'learning_rate': [0.1, 0.01, 0.05]}
# num_round = 5

# model = XGBClassifier(max_depth=7, learning_rate=1, objective='binary:logistic', enable_categorical=True, booster='gbtree')
# model.fit(X_train, y_train)
# preds = model.predict(X_test)
# print(preds.sum())
# print(len(preds))
# print(confusion_matrix(y_test, preds))
# print(classification_report(y_test, preds))
# print(f"roc auc: {sklearn.metrics.roc_auc_score( y_test, preds)}")

In [ ]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42,
    enable_categorical=True
)
parameters = {
    'max_depth': range (2, 20, 5),
    'n_estimators': range(10, 85, 5),
    'learning_rate': [0.1, 0.01, 0.05],
    'booster': ['gbtree'],
    'objective': ['binary:logistic']
}
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'aucpr',
    n_jobs = 10,
    #stratified
    cv = 10,
    verbose=10
)
grid_search.fit(X, y)
print(grid_search.best_estimator_)